# Pathology prediction (Lung opacity)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the lung opacity pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples :  0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value. 

The model is based on binary classification for each target chest pathology.

Final sample size for the Lung opacity pathology is : (N = 14,136)



#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.LUNG_OPACITY, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="LungOpacity", experiment_name="CP_LungOpacity", filepath="./results/lungopacity")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='kfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-10-02 10:30:37,949	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=289341) Outer fold 1
(run_fold pid=289351) Configuring PyCaret for outer fold 3
(run_fold pid=289351) Outer fold 3 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [04:22<01:37, 97.12s/it] [repeated 3x across cluster]


(run_fold pid=289351)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=289351) Fold                                                          
(run_fold pid=289351) 0       0.9718  0.8716  0.9994  0.9721  0.9855  0.4294  0.5134
(run_fold pid=289351) 1       0.9685  0.8389  0.9988  0.9693  0.9838  0.3763  0.4619
(run_fold pid=289351) 2       0.9701  0.7959  0.9988  0.9709  0.9847  0.4023  0.4830
(run_fold pid=289351) 3       0.9807  0.8101  1.0000  0.9805  0.9901  0.4996  0.5770
(run_fold pid=289351) 4       0.9751  0.8785  0.9994  0.9754  0.9873  0.4215  0.5058
(run_fold pid=289351) Mean    0.9733  0.8390  0.9993  0.9736  0.9863  0.4258  0.5082
(run_fold pid=289351) Std     0.0043  0.0326  0.0004  0.0040  0.0022  0.0412  0.0389
(run_fold pid=289332) Configuring PyCaret for outer fold 2 [repeated 2x across cluster]
(run_fold pid=289351) Tuning hyperparameters for model xgboost with custom grid using grid search


                                                         
Processing:  75%|███████▌  | 3/4 [04:24<01:37, 97.68s/it] [repeated 2x across cluster]


...

(run_fold pid=289332) Transformation Pipeline and Model Successfully Saved
(run_fold pid=289332)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=289332) 0  Extreme Gradient Boosting    0.9784  0.8641  ...  0.9889  0.5458  0.6076
(run_fold pid=289332) 
(run_fold pid=289332) [1 rows x 8 columns]
(run_fold pid=289332) Outer fold 5


Processing:   0%|          | 0/4 [00:00<?, ?it/s]


(run_fold pid=289332) Configuring PyCaret for outer fold 5


Processing:  75%|███████▌  | 3/4 [04:15<01:34, 94.49s/it]
                                                         


(run_fold pid=289332)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=289332) Fold                                                          
(run_fold pid=289332) 0       0.9724  0.8050  1.0000  0.9721  0.9859  0.3957  0.4966
(run_fold pid=289332) 1       0.9724  0.8414  1.0000  0.9721  0.9858  0.4466  0.5362
(run_fold pid=289332) 2       0.9701  0.8477  1.0000  0.9699  0.9847  0.3483  0.4592
(run_fold pid=289332) 3       0.9779  0.8070  0.9994  0.9783  0.9887  0.3865  0.4756
(run_fold pid=289332) 4       0.9751  0.8065  0.9994  0.9755  0.9873  0.3210  0.4218
(run_fold pid=289332) Mean    0.9736  0.8215  0.9998  0.9736  0.9865  0.3796  0.4779
(run_fold pid=289332) Std     0.0027  0.0189  0.0003  0.0030  0.0014  0.0429  0.0381
(run_fold pid=289332) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=289341) Transformation Pipeline and Model Successfully Saved
(run_fold pid=289341)                        Model  Accuracy  